#### This code gets all the monthly data from yahoo finance for a list of tickers.  We chose a monthly timeframe because a lot of the data we want to compare it is also in a monthly format.

In [83]:
    import datetime
    import time
    import urllib.request
    import pandas as pd

#### The following function retrieves all historical data in yahoo finance back to 1/1/1980 for a particular ticker.  It returns a dataframe for one ticker.  If data doesn't go back as far as 1980 for the ticker, data will be returned as far back as possible.  

In [84]:
def get_df_from_yahoo_finance (symbol): 

    end_date = datetime.datetime.now()
    end_date = int(round(end_date.timestamp())) * 1000  #<-- Convert datetime to milliseconds 
    end_date_str = str(end_date)
    end_date_str = end_date_str[:10]  #<-- This removes all but the last 10 millisecond values for the time which is format yahoo wants

    start_date_str = '345427200'
  

    ticker_dataframe = {}
    

    # Build Yahoo finance URL for the data request.
    
    yahoo_finance_url_with_symbol = ('https://query1.finance.yahoo.com/v7/finance/download/' + str(symbol)
                                     + '?period1=' + str(start_date_str) + '&period2=' + str(end_date_str)
                                     + '&interval=1mo&events=history&includeAdjustedClose=true')

    url2 = urllib.request.urlopen(yahoo_finance_url_with_symbol) #request data (comes as CSV)
    
    ticker_dataframe = pd.read_csv(url2)  #convert CSV in memory from the Yahoo request to dataframe
    ticker_dataframe.insert(1, "Ticker", symbol)

    return (ticker_dataframe)


#### This next section iterates through the list of tickers and appends them all to one dataframe as rows.  Add additonal tickers to the list_of_tickers_for_yahoo_data_dump list object.  

In [85]:

list_of_tickers_for_yahoo_data_dump = ['SPY', 'GLD', 'XLE', 'XLV']   # <-- Add tickers here

tickers_and_data_df = pd.DataFrame({})

for ticker in list_of_tickers_for_yahoo_data_dump:
    
    single_ticker_df = get_df_from_yahoo_finance(ticker)
    tickers_and_data_df = pd.concat([tickers_and_data_df, single_ticker_df])


#### Print the new dataframe --> tickers_and_data_df.

In [86]:
print (tickers_and_data_df)   #View the dataframe we just created

           Date Ticker        Open        High         Low       Close  \
0    1993-02-01    SPY   43.968750   45.125000   42.812500   44.406250   
1    1993-03-01    SPY   44.562500   45.843750   44.218750   45.187500   
2    1993-04-01    SPY   45.250000   45.250000   43.281250   44.031250   
3    1993-05-01    SPY   44.093750   45.656250   43.843750   45.218750   
4    1993-06-01    SPY   45.375000   45.812500   44.218750   45.062500   
..          ...    ...         ...         ...         ...         ...   
279  2022-04-01    XLV  137.369995  143.419998  130.089996  130.289993   
280  2022-05-01    XLV  130.179993  134.050003  125.279999  132.229996   
281  2022-06-01    XLV  132.710007  132.729996  118.750000  128.240005   
282  2022-07-01    XLV  128.289993  133.809998  125.230003  132.399994   
283  2022-08-01    XLV  131.660004  132.380005  131.119995  131.419998   

      Adj Close     Volume  
0     25.709480    5417600  
1     26.161797    3019200  
2     25.612762    26972

#### To view a dataframe of a single ticker use the following

In [87]:
spy_df = tickers_and_data_df.loc[tickers_and_data_df['Ticker'] == 'SPY']
print(spy_df)

           Date Ticker        Open        High         Low       Close  \
0    1993-02-01    SPY   43.968750   45.125000   42.812500   44.406250   
1    1993-03-01    SPY   44.562500   45.843750   44.218750   45.187500   
2    1993-04-01    SPY   45.250000   45.250000   43.281250   44.031250   
3    1993-05-01    SPY   44.093750   45.656250   43.843750   45.218750   
4    1993-06-01    SPY   45.375000   45.812500   44.218750   45.062500   
..          ...    ...         ...         ...         ...         ...   
350  2022-04-01    SPY  453.309998  457.829987  411.209991  412.000000   
351  2022-05-01    SPY  412.070007  429.660004  380.540009  412.929993   
352  2022-06-01    SPY  415.170013  417.440002  362.170013  377.250000   
353  2022-07-01    SPY  376.559998  413.029999  371.040009  411.989990   
354  2022-08-01    SPY  409.149994  413.410004  408.399994  410.769989   

      Adj Close      Volume  
0     25.709480     5417600  
1     26.161797     3019200  
2     25.612762     2